### Carrega dados da camada Bronze

In [0]:
from pyspark.sql import functions as f

In [0]:
aluno = spark.read.format("delta").load("/mnt/delta/bronze/sistema_logosofico/aluno")

### Realiza tratamento de dados

In [0]:
df = (
    aluno

    .withColumn("idade", 
                f.floor(f.months_between(f.current_date(), f.col("data_nascimento")) / 12))
    
    .withColumn("cpf", 
                f.regexp_replace(f.col("cpf"), r'(\d{3})\d{3}(\d{3})(\d{2})', r'\1***\2**'))
    
    .withColumn("email", 
                f.regexp_replace(f.col("email"), r'(.{2}).*@', r'\1***@'))
    
    .withColumn("telefone", 
                f.regexp_replace(f.col("telefone"), r'(\d{2})\d{3}(\d{4})', r'\1***\2'))
    
    .withColumn("endereco", 
                f.regexp_replace(f.col("endereco"), r'(.{5}).*', r'\1***'))
)

### Persiste dados tratados no Data Lake

In [0]:
df.write.format("delta").mode("overwrite").save(f"/mnt/delta/silver/academico/aluno")